In [1]:
import psycopg2
import sys
import pandas as pd

In [16]:
#Define our connection string
conn_string = "host='localhost' dbname='ohschools' user='postgres' password='dummy'"

# print the connection string we will use to connect
print("Connecting to database\n	{0}".format(conn_string))

# get a connection, if a connect cannot be made an exception will be raised here
conn = psycopg2.connect(conn_string)

# conn.cursor will return a cursor object, you can use this cursor to perform queries
cursor = conn.cursor()
print("Connected!\n")

Connecting to database
	host='localhost' dbname='ohschools' user='postgres' password='dummy'
Connected!



In [4]:
# Reads data from file
data = pd.read_csv('test.csv')
data.head()

,PCT_White,PCT_Black_or_AA,PCT_Asian,PCT_Hispanic_Latino,PCT_Less_Than_High_School,PCT_High_School_Grad,PCT_Bach_Or_Higher,Median_Income,PCT_Households_Poverty,PCT_Households_Limited_English,index
0,0.849407,0.083843,0.071616,0.02003,0.114222,0.599104,0.286674,40563,0.153846,0.0,0
1,0.971014,0.029851,0.000000,0.00000,0.201900,0.776722,0.021378,32045,0.534161,0.0,1
2,0.976000,0.000000,0.024590,0.00000,0.281167,0.527851,0.190981,44167,0.141914,0.0,2
3,0.933537,0.010451,0.000000,0.00000,0.063559,0.731992,0.204449,36759,0.233990,0.0,3
4,0.926049,0.079857,0.000000,0.00000,0.173184,0.702514,0.124302,49100,0.364000,0.0,4


In [11]:
# FUNCTION TO CREATE TABLE IN POSTGRES BASED ON PANDAS DF.
# MISSING PRIMARY KEY (PROBABLY FIRST VAR BY DEFAULT?)
def create_table(table_name, data):
    query = "CREATE TABLE IF NOT EXISTS {} (".format(table_name)
    for i, col in enumerate(data.columns):
        col_type=data.dtypes[i]
        # MORE TYPES, RAISE ERROR
        if col_type == 'int64':
            postgres_type = 'INT'
        else:# col_type == 'O':
            postgres_type = 'VARCHAR'
        string_var = "{} {} NOT NULL".format(col, postgres_type)
        
        if i==0:# Very ugly code (well, all the function is ugly but it works)
            query = query + string_var
        else:
            query = query + ", " + string_var
    query = query + ")"
    
    cursor.execute(query)
    conn.commit()

In [10]:
def upload_data(table_name, file_path):
    f = open(file_path, 'r')
    next(f)
    cursor.copy_from(f, table_name, sep=',')
    f.close()
    conn.commit()

In [17]:
create_table('census_test', data)

In [18]:
upload_data('census_test', 'test.csv')

In [19]:
# CHECK
cursor.execute("SELECT * from census_test;")
cursor.fetchall()

[('0.849406528189911',
  '0.08384279475982533',
  '0.07161572052401746',
  '0.020029673590504452',
  '0.11422172452407615',
  '0.5991041433370661',
  '0.2866741321388578',
  40563,
  '0.15384615384615385',
  '0.0',
  0),
 ('0.9710144927536232',
  '0.029850746268656716',
  '0.0',
  '0.0',
  '0.20190023752969122',
  '0.7767220902612827',
  '0.021377672209026127',
  32045,
  '0.5341614906832298',
  '0.0',
  1),
 ('0.976',
  '0.0',
  '0.02459016393442623',
  '0.0',
  '0.28116710875331563',
  '0.5278514588859416',
  '0.1909814323607427',
  44167,
  '0.1419141914191419',
  '0.0',
  2),
 ('0.9335365853658537',
  '0.010450685826257348',
  '0.0',
  '0.0',
  '0.0635593220338983',
  '0.7319915254237288',
  '0.2044491525423729',
  36759,
  '0.23399014778325122',
  '0.0',
  3),
 ('0.9260485651214128',
  '0.07985697258641239',
  '0.0',
  '0.0',
  '0.17318435754189945',
  '0.702513966480447',
  '0.12430167597765363',
  49100,
  '0.364',
  '0.0',
  4),
 ('0.9544117647058824',
  '0.029275808936825885',